In [63]:

with open("Sistemas de prueba\IEEE34.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()

# Extraer Lineas de las cargas en archivo
three_phase_loads = []
single_phase_loads = []
for idx, line in enumerate(lines_copy):
    if "C Load" in line:
        if "3f" in line:
            three_phase_loads.append(lines_copy[idx:idx+4])
            
        else:
            single_phase_loads.append(lines_copy[idx:idx+2])
            

In [65]:
# Iterar por cada Carga
from utils.components import single_phase_switch, three_phase_switch
for load_idx, loads in enumerate(three_phase_loads):
    print("\n")
    switch = three_phase_switch.copy()
    for idx, load in enumerate(loads):
        print(load, end='')
        
        if idx == 0:
            load_node = f"LN{load_idx}".center(6)
            continue

        node_1 = load[2:8]
        phase_1 = node_1.strip()[-1]
        print(phase)
        node_2 = load[8:14].strip()
        R = load[30:42]
        X = load[46:58]
        
        if node_2:
            # node_2 = node_2.center(6)
            phase_2 = node_2[-1]
        node_2 = node_2.center(6)
        switch_ln = switch[idx]
        switch[idx] = f"{switch_ln[:2]}{node_1}{load_node}{switch_ln[14:]}"
        loads[idx] = f"{load[:2]}{node_1}{node_2}{load[14:]}"
        print(switch[idx], end='')
        print(loads[idx], end='')

        





C Load 860 3fy
  X0072Aprueba                6300.9146341    5040.7317073                     0
A
  X0072A LN0         -1.      1.E3                                             0
  X0072Aprueba                6300.9146341    5040.7317073                     0
  X0072Bprueba                6300.9146341    5040.7317073                     0
B
  X0072B LN0         -1.      1.E3                                             0
  X0072Bprueba                6300.9146341    5040.7317073                     0
  X0072Cprueba                6300.9146341    5040.7317073                     0
C
  X0072C LN0         -1.      1.E3                                             0
  X0072Cprueba                6300.9146341    5040.7317073                     0


C Load 840 3fy
  N840A prueba                14307.923077    11128.384615                     0
A
  N840A  LN1         -1.      1.E3                                             0
  N840A prueba                14307.923077    11128.384615         

In [46]:
for loads in three_phase_loads:
    for load in loads:
        print(load, end='')

C Load 860 3fy
  X0072Aprueba 0
  X0072Bprueba 0
  X0072Cprueba 0
C Load 840 3fy
  N840A prueba 0
  N840B prueba 0
  N840C prueba 0
C Load 844 3f
  N844A prueba 0
  N844B prueba 0
  N844C prueba 0
C Load 830 3f
  N830A N830B  N830B N830C  N830C N830A20
C Load 848 3f
  N848A N848B10
  N848B N848C10
  N848C N848A10
C Load 802 3f
  N802A prueba 0
  N802B prueba 0
  N802C prueba 0
C Load 806 3f
  N806A prueba 0
  N806B prueba 0
  N806C prueba 0
C Load 836 3f a
  X0016AX0016B 0
  X0016BX0016C 0
  X0016CX0016A 0
C Load 840 3fd
  N840A N840B  N840B N840C40
  N840C N840AC Load 832 3f
  N832A N832B10
  N832B N832C  N832C N832AC Load 858 3f
  N358A N358B10
  N358B N358C  N358C N358AC Load 834 3fa
  N334A N334B  N334B N334C  N334C N334A90
C Load 860 3fd
  N860A N860B  N860B N860C  N860C N860A90
C Load 858 3fa
  X0005AX0005B 0
  X0005BX0005C 0
  X0005CX0005A 0
C Load 834 3f
  X0015AX0015B 0
  X0015BX0015C 0
  X0015CX0015A 0
C Load 860 3fa
  X0072AX0072B 0
  X0072BX0072C 0
  X0072CX0072A 0
C Load 8

In [69]:
phase_2 = None
phase_2 = ''
if phase_2:
    print('hola')